<a href="https://colab.research.google.com/github/3m6d/Computer_vision/blob/main/Week6_finetuning_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
import zipfile
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.optim as optim
import time

In [2]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.optim as optim
import time

In [3]:
cpu_count = os.cpu_count()

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
zip_path = '/content/drive/MyDrive/Cat_dog_images.zip'

In [6]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [7]:
data_dir = '/content/data/Cat_dog_images'

In [8]:
IMG_SIZE = 224

In [12]:
from torch.utils.data import random_split #random splits data

In [13]:
def get_data_loaders(data_dir, input_size=IMG_SIZE, batch_size=32, val_split=0.2 ):#validation split is 20%


# data augementation is where the data is changed by increasing the brightness
    transform = transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

  # in the following code, the data images is mapped for categorical values to index

    dataset = datasets.ImageFolder(data_dir, transform=transform) # dataset is torch.utils.data.Dataset object
    print("Dataset classes names:", dataset.classes)
    print("Classes and it's mapped index:", dataset.class_to_idx)
    print("")

    dataset_size = len(dataset)
    print("Total number of images in the dataset:", dataset_size)
    print("")

    val_size = int(val_split * dataset_size)
    train_size = dataset_size - val_size

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size]) # both are still PyTorch dataset object
    print("Train dataset size:", len(train_dataset))
    print("Validation dataset size:", len(val_dataset))
    print("")

    # The DataLoader is what actually feeds data into the model in batches during training.
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=cpu_count)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=cpu_count)

    print("Datatype of train_loader:", type(train_loader))
    inputs, labels = next(iter(train_loader))
    print(inputs.shape)
    print(labels.shape)

    dataloaders = {'train': train_loader, 'val': val_loader}
    dataset_sizes = {'train': train_size, 'val': val_size}
    class_names = dataset.classes

    return dataloaders, dataset_sizes, class_names

In [11]:
dipawoli = get_data_loaders(data_dir, input_size=IMG_SIZE, batch_size=32, val_split=0.2)

Dataset classes names: ['cats', 'dogs']
Classes and it's mapped index: {'cats': 0, 'dogs': 1}

Total number of images in the dataset: 210

Train dataset size: 168
Validation dataset size: 42

Datatype of train_loader: <class 'torch.utils.data.dataloader.DataLoader'>
torch.Size([32, 3, 224, 224])
torch.Size([32])


In [14]:
dataloaders, dataset_sizes, class_names = get_data_loaders(data_dir)

Dataset classes names: ['cats', 'dogs']
Classes and it's mapped index: {'cats': 0, 'dogs': 1}

Total number of images in the dataset: 210

Train dataset size: 168
Validation dataset size: 42

Datatype of train_loader: <class 'torch.utils.data.dataloader.DataLoader'>
torch.Size([32, 3, 224, 224])
torch.Size([32])


In [ ]:
# evry batch has 32 images with 3 channel with 224 by 224 size

In [15]:
def train_model(model, dataloaders, dataset_sizes, device, classification_type, num_epochs=10, lr=0.001):
    if classification_type == 1:
        criterion = nn.BCEWithLogitsLoss()
    else:
      criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=lr)

    model = model.to(device)

    for epoch in range(num_epochs):
        #Each epoch is one full pass over the dataset.
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 20)

        for phase in ['train', 'val']:
          # Two phases per epoch:
          # train: Model learns and updates weights.
          # val: Model evaluates accuracy on unseen data, no learning.

            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device).float().unsqueeze(1) # this is for binary classification for unsqueeze

                optimizer.zero_grad()
                # reset the gradient to zero
                # THIS MUST NOT BE AVOIDED!

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # forward prop
                    if classification_type == 1:
                        outputs = torch.sigmoid(outputs)
                        preds = (outputs > 0.5).float()
                    else:
                      _, preds = torch.max(outputs, 1)
                    # getting the predicted class and computing loss.

                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        # backpropagation and updating the parameters

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            # calculating each epoch loss and accuracy and normalizes by total number of sample.

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    return model

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [17]:
model_name = 'resnet18'  # Change as needed
if len(class_names) == 2:
    num_classes = 1
else:
    num_classes = len(class_names)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
# model = get_model(model_name, num_classes=num_classes)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 72.6MB/s]


In [18]:
trained_model = train_model(model, dataloaders, dataset_sizes, device, num_classes)

Epoch 1/10
--------------------
Train Loss: 0.6084 Acc: 0.8393
Val Loss: 0.6041 Acc: 0.7143
Epoch 2/10
--------------------
Train Loss: 0.5743 Acc: 0.8631
Val Loss: 0.6729 Acc: 0.7143
Epoch 3/10
--------------------
Train Loss: 0.5610 Acc: 0.8929
Val Loss: 0.5959 Acc: 0.7619
Epoch 4/10
--------------------
Train Loss: 0.5517 Acc: 0.9226
Val Loss: 0.7158 Acc: 0.6190
Epoch 5/10
--------------------
Train Loss: 0.5392 Acc: 0.9345
Val Loss: 0.7198 Acc: 0.6190
Epoch 6/10
--------------------
Train Loss: 0.5341 Acc: 0.9524
Val Loss: 0.5507 Acc: 0.8571
Epoch 7/10
--------------------
Train Loss: 0.5358 Acc: 0.9464
Val Loss: 0.5569 Acc: 0.8571
Epoch 8/10
--------------------
Train Loss: 0.5206 Acc: 0.9881
Val Loss: 0.5611 Acc: 0.8810
Epoch 9/10
--------------------
Train Loss: 0.5197 Acc: 0.9881
Val Loss: 0.5084 Acc: 0.9524
Epoch 10/10
--------------------
Train Loss: 0.5168 Acc: 1.0000
Val Loss: 0.5914 Acc: 0.6905
